# Домашнее задание 3

**Выполнил:** Веселов Илья Николаевич

**На сколько выполнил:** 3

**Правила игры:**

* возле каждой задачи указано число баллов (в сококупности можно получить 10 баллов)

* хотим чистый, читабельный код, сайты - с user-friendly интерфейсом)

* к каждому заданию должен быть приложен код проекта, выложенный на Github (если вы пользуетесь ScraperAPI или другими ключами, то ключи необходимо убирать)

* Оценивание будет происзодить в первую очередь по функциональности - все ли работает и работает так, как должно (но за полное отсутствие стиля мы будем снижать)

P.S. В данном задании вам не потребуется ScraperAPI :з

**Любые extra-вещи будут оцениваться в плюс как доп баллы**

## Задание 1 (3 балла)

Парсим [Steam](https://store.steampowered.com/) с помощью Scrapy!

Необходимо выбрать 3 запроса (например, "инди", "стратегии", "minecraft", на каждый запрос должно быть минимум 90 результатов), и вытащить по ним:

* Все игры на первых 2 страницах (найдите, каким образом отображаются страницы)

И для каждой игры вытащить:

* Название

* Категорию (весь путь, за исключением Все игры и самого названия игры)

* Число всех обзоров и общая оценка

* Дата выхода

* Разработчик

* Метки (тэги) игры

* Цена

* Доступные платформы

Сохраните игры в формате json, оставив только игры, выпущенные после 2000 года

Чтобы было проще, в репозитории с ДЗ выложены скрины, где можно посмотреть, где и какие поля находятся



In [2]:
import json

from scrapy import Item, signals, Spider, Field, Request
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings
from scrapy.signalmanager import dispatcher
from datetime import datetime

In [3]:
# список из запросов для поиска
queries = ['инди', 'стратегия', 'fps']

In [4]:
class Product(Item):
    url = Field()
    title = Field()
    category = Field()
    reviews_count = Field()
    rating = Field()
    release_date = Field()
    developer = Field()
    tags = Field()
    price = Field()
    platforms = Field()

    def to_dict(self):
        return {
            'url': self.get('url'),
            'title': self.get('title'),
            'category': self.get('category'),
            'reviews_count': self.get('reviews_count'),
            'rating': self.get('rating'),
            'release_date': self.get('release_date'),
            'developer': self.get('developer'),
            'tags': self.get('tags'),
            'price': self.get('price'),
            'platforms': self.get('platforms')
        }

    def get_release_year(self):
        try:
            return datetime.strptime(self.get('release_date'), '%d %b, %Y').year
        except ValueError:
            # иногда дата выхода игры - это некоторая строка, в которой нет года (например, Coming Soon)
            # в таком случае считаем, что игра нам подходит и вышла в текущем году
            return datetime.now().year

In [5]:
def digitify(string):
    """ Получаем строку и удаляем все, кроме цифр """
    return ''.join([ch for ch in string if ch.isdigit()])

In [6]:
class SteamSpider(Spider):
    name = 'steamparser'
    allowed_domains = ['store.steampowered.com']
    max_page = 2 # по заданию не нужно заходить дальше 2 страницы каталога

    def __init__(self, query, **kwargs):
        self.query = query
        self.start_urls = [f'https://store.steampowered.com/search/?term={self.query}']
        super().__init__(**kwargs)

    def start_requests(self):
        for page_num in range(1, self.max_page + 1):
            url = f'https://store.steampowered.com/search/?term={self.query}&page={page_num}'
            yield Request(url, callback=self.parse_catalog)

    def parse_catalog(self, response):
        for game_href in response.css('.search_result_row::attr("href")').extract():
            if not '/bundle/' in game_href:
                # если ссылка имеет вид *.com/bundle/* значит - это набор, а не игра
                yield Request(response.urljoin(game_href), callback=self.parse_game)

    def parse_game(self, response):
        
        title = response.css('#appHubAppName::text').extract_first('').strip()
        category = response.css('.blockbg').css('a::text').extract()[1:]
        reviews_count = response.css('.game_review_summary').xpath('..')
        reviews_count = reviews_count.css('.responsive_hidden::text').extract_first('')
        reviews_count = digitify(reviews_count)
        rating = response.css('.game_review_summary::text').extract_first('').strip()
        release_date = response.css('.date::text').extract_first('')
        developer = response.css('#developers_list').css('a::text').extract_first('').strip()
        tags = ','.join([tag.strip() for tag in response.css('.app_tag::text').extract()])
        price = response.css('[data-price-final]::attr("data-price-final")').extract_first('')
        try:
            price = float(price[:-2] + '.' + price[-2:])
        except ValueError:
            price = 'Бесплатно'
        platforms = response.css('[data-os]::attr("data-os")').extract()
        platforms = list(set(platforms))

        yield Product(
            url=response.request.url,
            title=title,
            category=category,
            reviews_count=reviews_count,
            rating=rating,
            release_date=release_date,
            developer=developer,
            tags=tags,
            price=price,
            platforms=platforms
        )

In [7]:
games = []

def crawler_results(signal, sender, item, response, spider):
    games.append(item)

dispatcher.connect(crawler_results, signal=signals.item_passed)

process = CrawlerProcess(get_project_settings())
process.settings.set('LOG_LEVEL', 'ERROR')
for query in queries:
    process.crawl(SteamSpider, query=query)
process.start()

INFO:scrapy.utils.log:Scrapy 2.7.1 started (bot: scrapybot)
2022-12-07 18:19:38 [scrapy.utils.log] INFO: Scrapy 2.7.1 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.0, Twisted 22.10.0, Python 3.8.15 (default, Oct 12 2022, 19:14:39) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 38.0.4, Platform Linux-5.10.133+-x86_64-with-glibc2.27
2022-12-07 18:19:38 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.0, Twisted 22.10.0, Python 3.8.15 (default, Oct 12 2022, 19:14:39) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 38.0.4, Platform Linux-5.10.133+-x86_64-with-glibc2.27
INFO:scrapy.crawler:Overridden settings:
{'LOG_LEVEL': 'ERROR'}
2022-12-07 18:19:38 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 'ERROR'}
/usr/local/lib/python3.8/dist-packages/scrapy/utils/request.py:231: ScrapyDeprecationWa

In [8]:
# убираем из результатов все, что было опубликовано до 2000 года
games = [game for game in games if game.get_release_year() > 2000]

# сохраняем в json файл
games = [game.to_dict() for game in games]
with open('games.json', 'w', encoding='utf-8') as f:
    json.dump(games, f, indent=4, ensure_ascii=False)

## Задание 2 (4 балла)

Создайте сайт с помощью Flask или Django. Будем делать конструктор резюме!

Общая структура сайта:

1. Страничка с авторизацией и регистрацией

2. После авторизации, должна быть страница с формой, в которой необходимо заполнить форму (если пользователь уже заполнял форму, то в формы уже должен быть подгружен текст, который был заполнен, для этого должна быть отдельная кнопочка для сохранения в черновик)

3. Страница со скоинструированным резюме, который может быть превращен в PDF и выгружен (для этого можно использовать [pdfkit](https://pypi.org/project/pdfkit/))

На всякий случай: пользователь не может посмотреть чужие резюме

## Задание 3* (2 балла)

Добавьте возможность загрузить картинку в резюме (для исходного задания это не требуется)

Разверните приложение на Yandex.Cloud или на любом другом хосте (если знаете куда можно)

## Задание 4 (3 балла)

Отрисуйте с помощью Dash или StreamLit данные из данного [датасета](https://www.kaggle.com/datasets/michaelbryantds/crimedata)

Здесь необходимо провести самостоятельный анализ данных (можно даже в ноутбуке), в качестве результата необходим дашборд, внутри которого будет:

* Не менее 5 селекторов

* Не менее 5 различных графиков, которые показывали бы зависимость между преступлениями (на ваш выбор, там можно выбрать) и выбранными вами показателями

Дашборд должен быть стилистически оформлен